<a href="https://colab.research.google.com/github/CharlesPrado23/Pos_Clustering_Modulo-I/blob/main/Collaborative_aula(NearestNeighbors_UserBased).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Collaborative Filtering

Estimated time needed: **25** minutes

## Objectives

After completing this lab you will be able to:

-   Create recommendation system based on collaborative filtering


Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.


<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Collaborative Filtering</a></li>
    </ol>
</div>
<br>
<hr>


<a id="ref1"></a>

# Acquiring the Data


To acquire and extract the data, simply run the following Bash scripts:  
Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). Lets download the dataset. To download the data, we will use **`!wget`** to download it from IBM Object Storage.  
**Did you know?** When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)


In [1]:
!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip

--2024-11-13 22:49:02--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  34.7MB/s    in 4.4s    

2024-11-13 22:49:07 (34.8 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv        

Now you're ready to start working with the data!


<hr>

<a id="ref2"></a>

# Preprocessing


First, let's get all of the imports out of the way:


In [2]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:


In [27]:
movies_df = pd.read_csv('movies.csv')

In [28]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [29]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv',usecols=['movieId','title'])
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'])

Let's also take a peek at how each of them are organized:


In [30]:
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


<br>


Next, let's look at the ratings dataframe.


In [31]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [32]:
res = movies_df[movies_df.title.str.contains('Star Wars')]
res

,movieId,title
257,260,Star Wars: Episode IV - A New Hope (1977)
1171,1196,Star Wars: Episode V - The Empire Strikes Back...
1184,1210,Star Wars: Episode VI - Return of the Jedi (1983)
2544,2628,Star Wars: Episode I - The Phantom Menace (1999)
5282,5378,Star Wars: Episode II - Attack of the Clones (...
10123,33493,Star Wars: Episode III - Revenge of the Sith (...
12943,61160,Star Wars: The Clone Wars (2008)
15535,79006,Empire of Dreams: The Story of the 'Star Wars'...
20456,100089,Star Wars Uncut: Director's Cut (2012)
23103,109713,Star Wars: Threads of Destiny (2014)


<hr>

<a id="ref3"></a>

# Collaborative Filtering


Now, time to start our work on recommendation systems.

The first technique we're going to take a look at is called **Collaborative Filtering**, which is also known as **User-User Filtering**. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the **Pearson Correlation Function**.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

The process for creating a User Based recommendation system is as follows:

-   Select a user with the movies the user has watched
-   Based on his rating to movies, find the top X neighbours
-   Get the watched movie record of the user for each neighbour.
-   Calculate a similarity score using some formula
-   Recommend the items with the highest score

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [33]:
userInput = [
            {'title':'Lord of the Rings: The Fellowship of the Ring, The (2001)', 'rating':5.0},
            {'title':'Star Wars: Episode VII - The Force Awakens (2015)', 'rating':4.5},
            {'title':'Matrix, The (1999)', 'rating':4.0},
            {'title':"Harry Potter and the Chamber of Secrets (2002)", 'rating':3.0},
            {'title':'V for Vendetta (2006)', 'rating':1.0},
            {'title':'Avengers: Age of Ultron (2015)', 'rating':4.5},
            {'title':'The Amazing Spider-Man 2 (2014)', 'rating':4.5},
            {'title':'Spider-Man 3 (2007)', 'rating':4.5}
         ]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Lord of the Rings: The Fellowship of the Ring,...",5.0
1,Star Wars: Episode VII - The Force Awakens (2015),4.5
2,"Matrix, The (1999)",4.0
3,Harry Potter and the Chamber of Secrets (2002),3.0
4,V for Vendetta (2006),1.0
5,Avengers: Age of Ultron (2015),4.5
6,The Amazing Spider-Man 2 (2014),4.5
7,Spider-Man 3 (2007),4.5


#### Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.


In [8]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe

#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,2571,"Matrix, The (1999)",4.0
1,4993,"Lord of the Rings: The Fellowship of the Ring,...",5.0
2,5816,Harry Potter and the Chamber of Secrets (2002),3.0
3,44191,V for Vendetta (2006),1.0
4,52722,Spider-Man 3 (2007),4.5
5,110553,The Amazing Spider-Man 2 (2014),4.5
6,122886,Star Wars: Episode VII - The Force Awakens (2015),4.5
7,122892,Avengers: Age of Ultron (2015),4.5


#### The users who has seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.


In [9]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
#userSubset = userSubset.sort_values(by=['rating'], ascending=False)

userSubset

,userId,movieId,rating
3,2,2571,3.5
105,4,2571,4.0
195,5,2571,2.5
254,7,122886,5.0
308,10,4993,4.0
...,...,...,...
22884215,247751,4993,4.0
22884267,247751,44191,4.0
22884341,247752,4993,0.5
22884364,247753,2571,4.0


In [10]:
userSubset.isna().sum()
len(userSubset)

158374

In [11]:
from scipy.sparse import csr_matrix
movies_users = userSubset.pivot_table(index = 'userId', columns='movieId',values='rating')
movies_users



movieId,2571,4993,5816,44191,52722,110553,122886,122892
userId,,,,,,,,
2,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
10,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
247745,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN
247750,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247751,5.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN


In [12]:
movies_users = movies_users.fillna(0)
movies_users

movieId,2571,4993,5816,44191,52722,110553,122886,122892
userId,,,,,,,,
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
10,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
247745,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0
247750,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
247751,5.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0


In [13]:
movies_users.index[0]

2

In [14]:
matrix_mv_users = csr_matrix(movies_users.values)
X = movies_users.values
X


array([[3.5, 0. , 0. , ..., 0. , 0. , 0. ],
       [4. , 0. , 0. , ..., 0. , 0. , 0. ],
       [2.5, 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [5. , 4. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0.5, 0. , ..., 0. , 0. , 0. ],
       [4. , 2.5, 0. , ..., 0. , 0. , 0. ]])

# NearestNeighbors
# USer-based

In [15]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(metric = 'cosine', algorithm='brute', n_neighbors=20).fit(X)

x_in = np.array(inputMovies.rating.tolist())
x_in = x_in.reshape(1,8)
distances, indices = nbrs.kneighbors(x_in)
indices[0]

array([76285, 24111, 54105, 58335, 70413, 28695, 58635, 61367,  5232,
       71885,  1603, 54646, 41612, 69312, 27012, 43879, 55763, 46070,
       10886,  1319])

In [16]:
distances[0]

array([0.00406901, 0.01047149, 0.01625669, 0.01699664, 0.01848435,
       0.02092523, 0.02135718, 0.02176802, 0.02220047, 0.02292066,
       0.02335323, 0.02376046, 0.02606222, 0.03041495, 0.03072075,
       0.03154829, 0.03271506, 0.03715683, 0.03737112, 0.03774796])

In [17]:
np.shape(distances)

(1, 20)

In [34]:
df_t = pd.DataFrame()
df_t['similarityIndex'] = 1/distances[0]
df_t['userId'] = movies_users.index[indices[0]]
df_t.head()

,similarityIndex,userId
0,245.759850,238366
1,95.497368,75057
2,61.513144,168834
3,58.835164,181986
4,54.099833,219745


In [35]:
df_t

,similarityIndex,userId
0,245.759850,238366
1,95.497368,75057
2,61.513144,168834
3,58.835164,181986
4,54.099833,219745
5,47.789192,89379
6,46.822655,182871
7,45.938943,191513
8,45.044098,16382
9,43.628760,224369


In [19]:
#!pip install fuzzywuzzy
#from fuzzywuzzy import process
#idx = process.extractOne('Toy Story',movies_df['title'])

#### The top x similar users to input user

Now let's get the top N users that are most similar to the input.


In [36]:
topUsers=df_t.sort_values(by='similarityIndex', ascending=False)
topUsers.head()
len(topUsers)

20

In [21]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head(20)

,similarityIndex,userId,movieId,rating
0,245.75985,238366,1,4.0
1,245.75985,238366,21,3.0
2,245.75985,238366,31,4.0
3,245.75985,238366,47,3.5
4,245.75985,238366,65,3.5
5,245.75985,238366,70,4.5
6,245.75985,238366,95,3.5
7,245.75985,238366,104,4.0
8,245.75985,238366,110,3.5
9,245.75985,238366,141,1.0


In [22]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,245.75985,238366,1,4.0,983.039401
1,245.75985,238366,21,3.0,737.279551
2,245.75985,238366,31,4.0,983.039401
3,245.75985,238366,47,3.5,860.159476
4,245.75985,238366,65,3.5,860.159476


In [23]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,966.009813,3943.829077
2,570.191220,1920.701273
3,160.444978,409.799334
5,221.495862,661.091390
6,462.522202,2027.146396


In [24]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,4.082597,1
2,3.368521,2
3,2.554142,3
5,2.984667,5
6,4.382809,6


Now let's sort it and see the top 20 movies that the algorithm recommended!


In [25]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
140341,5.0,140341
146196,5.0,146196
132632,5.0,132632
84844,5.0,84844
125972,5.0,125972


In [26]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(20)['movieId'].tolist())]

,movieId,title
1798,1881,Quest for Camelot (1998)
3201,3287,"Tigger Movie, The (2000)"
3203,3289,Not One Less (Yi ge dou bu neng shao) (1999)
7043,7154,Mona Lisa Smile (2003)
7104,7215,To Have and Have Not (1944)
8516,25961,"Gunfighter, The (1950)"
12891,60806,"Little Mermaid 2: Return to the Sea, The (2000)"
13110,62970,Tin Man (2007)
13919,69685,Daria: Is It College Yet? (2002)
15059,76111,About Elly (Darbareye Elly) (2009)


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages

-   Takes other user's ratings into consideration
-   Doesn't need to study or extract information from the recommended item
-   Adapts to the user's interests which might change over time

##### Disadvantages

-   Approximation function can be slow
-   There might be a low of amount of users to approximate
-   Privacy issues when trying to learn the user's preferences


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="https://www.ibm.com/analytics/spss-statistics-software">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://www.ibm.com/cloud/watson-studio">Watson Studio</a>


### Thank you for completing this lab!

## Author

Saeed Aghabozorgi

### Other Contributors

<a href="https://www.linkedin.com/in/joseph-s-50398b136/" target="_blank">Joseph Santarcangelo</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2020-11-03        | 2.1     | Lakshmi    | Updated URL of csv                 |
| 2020-08-27        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |
|                   |         |            |                                    |
|                   |         |            |                                    |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
